# Physics Ball Game

Try out this physics ball game using pygame!


**How it works:**
1. Start of with one ball
2. Add more balls using the buttons
3. Press space to boost all balls up
4. Press esc to quit


**Required modules:**
- `pip install pygame`

In [ ]:
import pygame
import sys
import random
import math

# Initialize Pygame
pygame.init()

# Dimensions
SIDEBAR_WIDTH = 220
GAME_WIDTH, HEIGHT = 800, 600
WIDTH = GAME_WIDTH + SIDEBAR_WIDTH

# Screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pysurf = screen 
pygame.display.set_caption("Physics Ball Game ")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (50, 100, 255)
LIGHT_GRAY = (235, 235, 235)
GRAY = (200, 200, 200)
DARK_GRAY = (150, 150, 150)
SEPARATOR = (220, 220, 220)

# Physics
gravity = 0.5
bounce_factor = -0.7

# Font
FONT = pygame.font.SysFont(None, 24)
TITLE_FONT = pygame.font.SysFont(None, 28)

# Clock
clock = pygame.time.Clock()
FPS = 60


class Ball:
    def __init__(self, x, y, vx, vy, radius=20, color=BLUE, random_motion=True):
        self.x = float(x)
        self.y = float(y)
        self.vx = float(vx)
        self.vy = float(vy)
        self.r = int(radius)
        self.color = color
        self.random_motion = random_motion

    def update(self):

        # Apply gravity
        self.vy += gravity

        # Update position
        self.x += self.vx
        self.y += self.vy

        # Collisions with floor/ceiling
        if self.y + self.r > HEIGHT:
            self.y = HEIGHT - self.r
            self.vy *= bounce_factor
        if self.y - self.r < 0:
            self.y = self.r
            self.vy *= bounce_factor

        # Collisions with left/right walls (game area only)
        if self.x + self.r > GAME_WIDTH:
            self.x = GAME_WIDTH - self.r
            self.vx *= -1
        if self.x - self.r < 0:
            self.x = self.r
            self.vx *= -1

    def draw(self, surf):
        pygame.draw.circle(surf, self.color, (int(self.x), int(self.y)), self.r)


class Button:
    def __init__(self, x, y, w, h, text, color=GRAY, hover=DARK_GRAY, text_color=BLACK):
        self.rect = pygame.Rect(x, y, w, h)
        self.text = text
        self.color = color
        self.hover_color = hover
        self.text_color = text_color

    def draw(self, surf):
        hovered = self.rect.collidepoint(pygame.mouse.get_pos())
        pygame.draw.rect(surf, self.hover_color if hovered else self.color, self.rect, border_radius=8)
        label = FONT.render(self.text, True, self.text_color)
        label_rect = label.get_rect(center=self.rect.center)
        surf.blit(label, label_rect)

    def is_clicked(self, event):
        return (
            event.type == pygame.MOUSEBUTTONDOWN
            and event.button == 1
            and self.rect.collidepoint(event.pos)
        )


# Buttons in sidebar
PAD = 16
BTN_W = SIDEBAR_WIDTH - 2 * PAD
BTN_H = 44
SIDEX = GAME_WIDTH + PAD

add1_btn = Button(SIDEX, 20, BTN_W, BTN_H, "+ Add Random Ball")
add5_btn = Button(SIDEX, 20 + (BTN_H + 12) * 1, BTN_W, BTN_H, "+ Add 5 Balls")
clear_btn = Button(SIDEX, 20 + (BTN_H + 12) * 2, BTN_W, BTN_H, "Clear Balls")

def random_color():
    return (
        random.randint(30, 255),
        random.randint(30, 255),
        random.randint(30, 255),
    )


def add_random_balls(container, n=1):
    for _ in range(n):
        r = random.randint(12, 24)
        x = random.uniform(r, GAME_WIDTH - r)
        y = random.uniform(r, HEIGHT / 2)  # spawn in upper half
        vx = random.uniform(-4, 4)
        vy = random.uniform(-2, 2)
        color = random_color()
        container.append(Ball(x, y, vx, vy, r, color, random_motion=True))


# --- Ball-Ball Collision Physics ---
def handle_collisions(balls, restitution=0.9):
    n = len(balls)
    if n < 2:
        return
    for i in range(n):
        for j in range(i + 1, n):
            b1 = balls[i]
            b2 = balls[j]

            dx = b2.x - b1.x
            dy = b2.y - b1.y
            dist_sq = dx * dx + dy * dy
            r_sum = b1.r + b2.r

            # Check overlap
            if dist_sq <= 0:
                # Avoid divide-by-zero; nudge slightly
                dx, dy = 0.01, 0.0
                dist_sq = dx * dx + dy * dy

            if dist_sq < r_sum * r_sum:
                dist = math.sqrt(dist_sq)
                # Collision normal
                nx = dx / dist
                ny = dy / dist

                # Mass proportional to area (r^2)
                m1 = float(b1.r * b1.r)
                m2 = float(b2.r * b2.r)
                total_mass = m1 + m2

                # Positional correction (separate the balls)
                overlap = (r_sum - dist)
                if total_mass > 0:
                    b1.x -= nx * (overlap * (m2 / total_mass))
                    b1.y -= ny * (overlap * (m2 / total_mass))
                    b2.x += nx * (overlap * (m1 / total_mass))
                    b2.y += ny * (overlap * (m1 / total_mass))
                else:
                    b1.x -= nx * (overlap * 0.5)
                    b1.y -= ny * (overlap * 0.5)
                    b2.x += nx * (overlap * 0.5)
                    b2.y += ny * (overlap * 0.5)

                # Relative velocity
                rvx = b1.vx - b2.vx
                rvy = b1.vy - b2.vy
                rvn = rvx * nx + rvy * ny

                # If moving apart, skip impulse
                if rvn > 0:
                    continue

                # Compute impulse scalar for 1D along normal
                j = -(1.0 + restitution) * rvn
                j /= (1.0 / m1 + 1.0 / m2)

                # Apply impulse
                impulse_x = j * nx
                impulse_y = j * ny
                b1.vx += impulse_x / m1
                b1.vy += impulse_y / m1
                b2.vx -= impulse_x / m2
                b2.vy -= impulse_y / m2

# Ball list, seed with one ball for continuity
balls = []
add_random_balls(balls, 1)


running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
            running = False

        # Buttons
        if add1_btn.is_clicked(event):
            add_random_balls(balls, 1)
        elif add5_btn.is_clicked(event):
            add_random_balls(balls, 5)
        elif clear_btn.is_clicked(event):
            balls.clear()

    # Key hold: SPACE to give upward boost to all balls
    keys = pygame.key.get_pressed()
    if keys[pygame.K_SPACE]:
        for b in balls:
            b.vy = -10

    # Update physics
    for b in balls:
        b.update()
    # Resolve ball-ball collisions
    handle_collisions(balls)

    # Draw frame
    screen.fill(WHITE)

    # Separator and sidebar background
    pygame.draw.rect(screen, LIGHT_GRAY, (GAME_WIDTH, 0, SIDEBAR_WIDTH, HEIGHT))
    pygame.draw.line(screen, SEPARATOR, (GAME_WIDTH, 0), (GAME_WIDTH, HEIGHT), 2)

    # Draw balls in game area
    for b in balls:
        b.draw(screen)

    # Sidebar contents
    title = TITLE_FONT.render("Controls", True, BLACK)
    screen.blit(title, (SIDEX, 20 + -28))

    add1_btn.draw(screen)
    add5_btn.draw(screen)
    clear_btn.draw(screen)

    hint1 = FONT.render("SPACE = boost up", True, BLACK)
    hint2 = FONT.render("ESC = quit", True, BLACK)
    screen.blit(hint1, (SIDEX, HEIGHT - 60))
    screen.blit(hint2, (SIDEX, HEIGHT - 36))

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
sys.exit()

